In [ ]:
# Mounting your Google Drive in the runtime's virtual machine

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install Python Modules
! pip install geemap
! pip install whiteboxgui

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 8.6 MB/s eta 0:00:00


In [ ]:
## Import Python Modules

import ee
import geemap as geemap
import whiteboxgui as whitebox
import os
import numpy as np

Decompressing WhiteboxTools_linux_musl.zip ...
WhiteboxTools package directory: /usr/local/lib/python3.10/dist-packages/whitebox


In [ ]:
## Open a folium map object with 'geemap'

Map = geemap.Map()

In [ ]:
# Set your ROI (Region Of Interest)

ROI = ee.Geometry.Polygon(
[[22.67838587841031,37.82710900774967],
[22.67838587841031,37.86289590066289],
[22.626887465324373,37.86289590066289],
[22.626887465324373,37.82710900774967]], 'EPSG:4326', False)


In [ ]:
## To customise the AOI, run this cell and follow the instructions in the next cell.

Map.centerObject(ROI, 4)
Map

Map(center=[37.84500194323629, 22.652636671863878], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
## To customise the AOI, draw a polygon in the map above and run this cell.
CustomDraw = ee.Feature(Map.draw_last_feature)
Geom = CustomDraw.geometry()
Coords = Geom.coordinates()
ROI = ee.Geometry.Polygon(Coords)
#print(ROI)

In [ ]:
## This is the cloud masking function provided by GEE but adapted for use in Python.

def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  
  return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image,["system:time_start"])

In [ ]:
# S2 imqge collection

S2_Period = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterBounds(ROI)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
          .select(['B2','B3','B4','QA60'])
          .filter(ee.Filter.dayOfYear(1,360)))
 
S2_coll = (S2_Period.map(maskS2clouds))

In [ ]:
## Reducing the images collection in a single 6-bands image

S2_RGB = (S2_Period.select(['B2','B3','B4']).median().clip(ROI))

In [ ]:
## RGB composite

## bands 4–3–2 (Red/Green/Blue), visualisation parameters:

RGB_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': [191.501, 265.506, 324.995],
    'max': [1952.89, 1550.66, 1178],
  }

In [ ]:
Alos_dem = ee.Image("JAXA/ALOS/AW3D30/V2_2").select('AVE_DSM')
vis_params = {
  'min': 50,
  'max': 1000,
  'palette': ['5AB4AC', 'F5F5F5', 'D8B365']}
Clipped_dem1 = Alos_dem.clip(ROI)

In [ ]:
srtm_dem = ee.Image('USGS/SRTMGL1_003').select('elevation')
vis_params = {
  'min': 50,
  'max': 1000,
  'palette': ['5AB4AC', 'F5F5F5', 'D8B365']}
Clipped_dem2 = srtm_dem.clip(ROI)

In [ ]:
#Add Layers
#Map.add_basemap(basemap='SATELLITE')
Map.addLayer(ROI, {},'ROI')
Map.addLayer(Clipped_dem1, vis_params,'ALOS 30_DEM', True)
Map.addLayer(Clipped_dem2, vis_params,'SRTM 30_DEM', True)
Map.addLayer(S2_RGB, RGB_vis, "RGB", True)

In [ ]:
# Set map center
Map.center_object(Clipped_dem1, zoom = 16) 

In [ ]:
Map.add_layer_control()
Map

Map(bottom=748992.0, center=[45.67686103703512, 8.233308499975278], controls=(WidgetControl(options=['position…

In [ ]:
#Set Coordinates Reference Systems (CRS), For example: Italy --> 'EPSG:32632';  WGS 84 --> 'EPSG:4326'; Oman : 32640

pjcrs = 'EPSG:32632'

In [ ]:
## Download image and save it to sample_data in your Drive
## NB: Select the ROI's corresponding CRS! For example: Italy --> 'EPSG:32632'; Iraq --> 'EPSG:32638';  WGS 84 --> 'EPSG:4326'.

geemap.ee_export_image(ee_object= S2_RGB, filename='RGB.tif',scale=10,crs= pjcrs,region= ROI, file_per_band= False)
geemap.ee_export_image(ee_object= Clipped_dem1, filename='ALOS 30_DEM.tif',scale=30,crs= pjcrs,region= ROI, file_per_band= False)
geemap.ee_export_image(ee_object= Clipped_dem2, filename='SRTM 30_DEM.tif',scale=30,crs=pjcrs,region= ROI, file_per_band= False)

Generating URL ...
An error occurred while downloading.
Total request size (321427251 bytes) must be less than or equal to 50331648 bytes.
Generating URL ...
Please wait ...
Data downloaded to /content/ALOS 30_DEM.tif
Generating URL ...
Please wait ...
Data downloaded to /content/SRTM 30_DEM.tif
